<a href="https://colab.research.google.com/github/priigimenez/genai-foundations/blob/main/lab02_Intelligent_Triage_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intelligent Triage System & Customer Experience (CX) Optimization

El propósito de este módulo es diseñar e implementar una capa de Inteligencia operativa capaz de procesar feedback público (reviews de Google) y transformarlo en datos estructurados y accionables.

**Buscamos optimizar el flujo de trabajo de los equipos de Soporte y Producto, permitiendo una respuesta inmediata ante incidentes críticos.**

## Fundamentación técnica


En entornos de alto volumen, la lectura manual de miles de reseñas diarias es ineficiente. Este sistema utiliza GenAI para realizar tres tareas simultáneas en milisegundos:

- Clasificación automática: categorización de reclamos (Onboarding, Pagos, Tarjetas, etc).

- Análisis de sentimiento y priorización: identificación del nivel de urgencia y frustración del usuario para escalar casos críticos (fraude o pérdida de fondos).

- Extracción de entidades: identificación de montos, fechas y puntos de dolor específicos.

#### *Nota de arquitectura*
Aunque este laboratorio trabaja con datos públicos, el sistema está diseñado bajo el principio de Modularidad. En un flujo de producción que incluya datos privados (chats internos), se integraría la capa de seguridad (lab 01) como paso previo para garantizar el uso adecuado de la información personal.

## Tecnologías aplicadas
- Modelo: gemini-flash-lite-latest (optimizado para latencia y costo).
- Formato de salida: JSON Schema (para integración directa con APIs y bases de datos).
- Técnica: Prompt Engineering para extracción de datos estructurados.

In [ ]:
!pip install -q -U google-genai

In [2]:
import os
from google import genai
from google.genai import types
from google.colab import userdata

client = genai.Client(
    api_key=userdata.get("GOOGLE_API_KEY")
)


## Konwledge base

In [3]:
# Definimos las categorías oficiales para el Triage
categorias_quejas = """
CATEGORÍAS DE SOPORTE:
1. TARJETAS: Problemas con envío, activación, pérdida o cobros indebidos con la tarjeta física o virtual.
2. PAGOS/TRANSFERENCIAS: Dinero que no llegó, CBU/CVU, pagos de servicios o recargas.
3. ONBOARDING: Problemas al registrarse, selfie rechazada o validación de DNI.
4. CUENTA/APP: Error de login, congelamiento de cuenta o problemas de interfaz.

PRIORIDADES:
- ALTA: Fraude, cuenta bloqueada con saldo, pérdida de tarjeta.
- MEDIA: Transferencia demorada, error en pago de servicios.
- BAJA: Consultas informativas, sugerencias, problemas estéticos de la app.
"""

In [4]:
# Dataset basado en problemas reales reportados por usuarios
reseñas_criticas = [
    "Pésimo servicio, hace 3 días mandé plata a mi cuenta de banco y desapareció. En el chat no contesta nadie y la plata no llega. Son unos chorros.",
    "No me deja entrar a la app. Pongo mi clave y se queda la pantalla en blanco. Ya re instale todo y sigue igual, arreglen esto que tengo que pagar el alquiler.",
    "Me bloquearon la cuenta por seguridad dicen pero es mi plata. Mande la selfie 10 veces y me la rechaza. Atencion al cliente un desastre.",
    "Quise pagar el telecentro y me dio error pero la plata me la descontaron igual. Ahora quien me devuelve los 8000 pesos???",
    "Pésima billetera virtual, increíble que te retenga plata y que te haga débitos injustificados. No la instalen y menos pongan plata en ella.",
    "La verdad son un desastre me bloquearon mi cuenta siendo monotributista estando anotado teniendo posnet teniendo el plugin para e-commerce pagando todos los meses como corresponde y contribuyendo para que me hayan retenido plata diciendo de que por seguridad y creen que soy un estafador por lo que me dieron a entender con mails 🤣",
    "Muy mal servicio. Me dejaron una transferencia pendiente y el dinero colgado. Un bot mugriento responde, pero no personas. Envié emails, pero sigo sin mi dinero. Si logro recuperar el dinero, cierro la cuenta. No recomiendo Ualá.",
    "hace una semana no puedo retirar mi dinero. La ayuda que te ofrecen es un bot asqueroso que te deriva a un agente. El agente (podria ser otro bot) no sabe porque suceden los errores y te dice 'estamos trabajando'. que infumable toda la situacion.",
    "Hace varios días que no puedo ingresar desde la app, no desde la página por Google, me dice que hay un error que intente más tarde. Ya envié varios mails para recuperar el usuario y nadie contesta. Necesito una solución"

]

## Implementación del Triage

In [5]:
def triage_inteligente(reseña):
    # Prompt optimizado para clasificación y extracción pura
    prompt_sistema = f"""
    Eres un Analista de Operaciones. Tu tarea es procesar el feedback público y extraer datos estructurados.

    CATEGORÍAS: {categorias_quejas}

    RESPONDE EXCLUSIVAMENTE EN FORMATO JSON:
    {{
      "categoria": "TARJETAS | PAGOS | ONBOARDING | CUENTA",
      "prioridad": "ALTA | MEDIA | BAJA",
      "sentimiento": "Positivo | Neutral | Negativo | Muy negativo",
      "resumen_agente": "Breve descripción técnica",
      "accion_sugerida": "Paso a seguir por el equipo de soporte"
    }}
    """

    response = client.models.generate_content(
        model="gemini-flash-lite-latest",
        contents=f"{prompt_sistema}\n\nRESEÑA: {reseña}",
        config={"response_mime_type": "application/json"}
    )
    return response.text

# Ejecutamos con tus ejemplos reales
for r in reseñas_criticas:
    resultado = triage_inteligente(r)
    print(f"Reseña: {r[:70]}...")
    print(f"Análisis: {resultado}\n")

Reseña: Pésimo servicio, hace 3 días mandé plata a mi cuenta de banco y desapa...
Análisis: {
  "categoria": "PAGOS/TRANSFERENCIAS",
  "prioridad": "ALTA",
  "sentimiento": "Muy negativo",
  "resumen_agente": "Transferencia enviada hace 3 días no acreditada en cuenta de destino. El canal de chat no responde.",
  "accion_sugerida": "Investigar inmediatamente el estado de la transferencia usando los datos de origen y destino, y contactar al usuario proactivamente ofreciendo una solución o tracking claro."
}

Reseña: No me deja entrar a la app. Pongo mi clave y se queda la pantalla en b...
Análisis: {
  "categoria": "CUENTA/APP",
  "prioridad": "ALTA",
  "sentimiento": "Negativo",
  "resumen_agente": "El usuario reporta imposibilidad de acceso a la aplicación tras ingresar la clave, la pantalla queda en blanco. Reinstalación no solucionó el problema. Indica urgencia por necesidad de realizar un pago importante (alquiler).",
  "accion_sugerida": "Escalar a soporte técnico nivel 2 para revi

## Conclusión y próximos pasos


**Escalabilidad y Automatización**

Para este lab, utilizamos una muestra de datos reales mediante manual scraping. Sin embargo, el diseño es compatible con una arquitectura de Ingesta en tiempo real. Mediante el uso de herramientas como Selenium o APIs de monitoreo de tiendas (Google Play/App Store), el sistema puede automatizar la captura de feedback, procesándolo de forma asincrónica a través de este pipeline de GenAI para alimentar tableros de control de salud del producto (Product Health Dashboards)."